<div class='alert alert-success'>
    <h1>3D Demonstrations</h1>
    <h3>Yen Lee Loh</h3>
    <h4>2023-1-17 </h4>
</div>

In [1]:
import importlib; import threed; importlib.reload (threed); from threed import *
import itertools
def dump(expr):
  if isinstance(expr, np.ndarray) and len(expr.shape) >= 2:
    print (expr, '=\n', eval(expr))
  else:
    print (expr, '=', eval(expr))    

In [2]:
#======== Draw a dumbbell using functions supplied by Loh's threed wrapper
objects = []
objects.append( sphere([1,1,1], 0.5,color='#99FFFF') )
objects.append( sphere([2,2,2], 0.5,color='#FFFF99') )
objects.append( cylinder([1,1,1], [2,2,2], 0.2, color='#99FF99') )
scene = Scene(children=[*objects])
scene

Preview(child=Scene(children=(Mesh(geometry=SphereBufferGeometry(heightSegments=24, widthSegments=24), materia…

In [3]:
#======== If we want more control over the result, we have to supply more details ourselves
render(objects, cameraPosition=[0,-1,0], viewCenter=[1.5,1.5,1.5], imageSize=[512,384])

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

In [18]:
#======== DRAW A COLLECTION OF ATOMS WITH AUTOMATIC BONDS
#rad = np.array([[0,0,0],[1,0,0],[-1,0,0],[0,1,0],[0,-1,0]])
rad = np.array([[0,0,0],[1,0,0],[0,1,0],[0,0,1],[0,1,1],[1,1,0],[1,0,1],[1,1,1]]) * 5
#rng = np.random.default_rng ()
#rad = rng.uniform (low=0, high=5, size=[9,3])
objects = []
for rd in rad:
  objects.append ( sphere (rd, radius=1, color='#9999FF') )
bondLengths = [np.linalg.norm(rdB-rdA) for rdA,rdB in itertools.combinations(rad, 2)]
maxLength = np.min (bondLengths) * 1.01
for rdA,rdB in itertools.combinations (rad, 2):
  if np.linalg.norm(rdB-rdA) < maxLength:
    objects.append ( cylinder (rdA, rdB, radius=.2, color='#99FF99') )
  
#======== DETERMINE viewCenter AND cameraPosition AUTOMATICALLY
elev = 30 * np.pi/180
azim = 5 * np.pi/180
viewCenter = np.mean (rad, axis=0)
viewExtent = np.max (np.linalg.norm(rad - viewCenter, axis=0))
viewFoV = 50 * np.pi/180
viewDistance = viewExtent  / np.tan (viewFoV/2)
cameraPosition = viewCenter + viewDistance * np.array([np.cos(elev)*np.cos(azim),np.cos(elev)*np.sin(azim),np.sin(elev)]) # not sure about math
#======== DETERMINE viewCenter AND cameraPosition AUTOMATICALLY
render (objects, imageSize=[512,384], cameraPosition=tuple(cameraPosition), fov=viewFoV*180/np.pi, viewCenter=tuple(viewCenter))

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

In [57]:
#======== DRAW A UNIT CELL (WHICH DOUBLES UP AS A BOUNDING BOX)
Jld = np.diag([5,5,1])
objects = []
rnd = np.array([[0,0,0],[1,0,0],[0,1,0],[0,0,1],[0,1,1],[1,1,0],[1,0,1],[1,1,1]]) @ Jld # bounding box corners
rdCenter = np.mean (rnd, axis=0)     # view center
for rd in rnd:
  objects.append ( sphere (rd, radius=.05, color='#9999FF') ) # mark each corner
for u in [0,1]:
  for v in [0,1]:
    for w in [0,1,2]:
      rdA = np.roll ([u,v,0], w) @ Jld
      rdB = np.roll ([u,v,1], w) @ Jld
      objects.append ( cylinder (rdA, rdB, radius=.2, color='#99FF99') )
      
#======== GIVEN THESE PARAMETERS
width,height = 512,384  # pixels
elev,azim = 60,45       # degrees
fov = 50                # degrees
cameraUp = [0,0,1] 
#======== DETERMINE viewCenter AND cameraPosition AUTOMATICALLY TO FIT SCENE IN VIEW
e=elev*np.pi/180; a=azim*np.pi/180
cameraFw = -np.array([np.cos(e)*np.cos(a),np.cos(e)*np.sin(a),np.sin(e)])
JldCam = lookAt (cameraFw, cameraUp)
unl = (rnd - rdCenter) @ JldCam.T
beta = fov*np.pi/180; q = 1/np.tan(beta/2); p = 1/np.tan(beta/2)*width/height
viewDistance = max( [w + max(p*abs(u), q*abs(v)) for u,v,w in unl] )
 
dump ('rdCenter')
dump ('viewDistance')
cameraPosition = rdCenter - viewDistance * cameraFw
render (objects, imageSize=[width,height], cameraPosition=tuple(cameraPosition), fov=fov, viewCenter=tuple(rdCenter))

rdCenter = [2.5 2.5 0.5]
viewDistance = 10.754560133614763


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

rdCenter = [2.5 2.5 0.5]
viewDistance = 36.24178200849264


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

For a perspective camera, we are given the vertical angular field of view (FoV) $\beta$, which is typically 50 degrees, and the aspect ratio of the camera view $Y/X$.
Let $q = \cot\frac{\beta}{2}$ and $p = \frac{X}{Y} \cot\frac{\beta}{2}$.

Now, take the bounding box corners $r_{nd}$ and perform the following.
1. Translate so that view center is at origin.
2. Rotate about origin so that viewing direction is (0,0,-1).
3. For each coordinate $(x_d,y_d,z_d)$, calculate $z_n + \max (p \left|x_n\right|, q \left|y_n\right|)$
4. Set the view distance to be $z = \max_n z_n$.  That is, put the camera position at CoM - ...

In [30]:
abs(-3.4)

3.4

In [6]:
= 
Xn =m


viewExtent = np.max (np.linalg.norm(rad - viewCenter, axis=0))
viewFoV = 50 * np.pi/180
viewDistance = viewExtent  / np.tan (viewFoV/2)
cameraPosition = viewCenter + viewDistance * np.array([np.cos(elev)*np.cos(azim),np.cos(elev)*np.sin(azim),np.sin(elev)]) # not sure about math
#======== DETERMINE viewCenter AND cameraPosition AUTOMATICALLY
render (objects, imageSize=[512,384], cameraPosition=tuple(cameraPosition), fov=viewFoV*180/np.pi, viewCenter=tuple(viewCenter))

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

<div class="alert alert-info"><h2>  Trying to convert 3D display to image (difficult(  </h2></div>

In [5]:
geom = CylinderBufferGeometry(0,5,24,24)
geom

Preview(child=CylinderBufferGeometry(height=24.0, radialSegments=24, radiusBottom=5.0, radiusTop=0.0), shadowM…

In [7]:
geom.rotateZ()

AttributeError: 'CylinderBufferGeometry' object has no attribute 'rotateZ'

In [8]:
geom.exec_three_obj_method ('rotateZ', np.pi/2)
geom

Preview(child=CylinderBufferGeometry(height=24.0, radialSegments=24, radiusBottom=5.0, radiusTop=0.0), shadowM…

In [110]:
#help (RenderableWidget)

In [67]:
sphere = Mesh(SphereBufferGeometry(1,24,24), MeshPhongMaterial(color='#660000'))
camera = PerspectiveCamera( position=[0, 0, 5], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()
scene = Scene(children=[sphere, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)
renderTarget = WebGLRenderTarget()
renderer.exec_three_obj_method (setRenderTarget, renderTarget)
renderer


AttributeError: 'Renderer' object has no attribute 'exec_three_obj_method'

In [ ]:
  renderer.setRenderTarget(renderTarget);
  renderer.render(rtScene, rtCamera);

In [53]:
renderTarget = WebGLRenderTarget()
renderTarget

Preview(child=WebGLRenderTarget(), shadowMap=WebGLShadowMap())

In [ ]:

 |  exec_three_obj_method(self, method_name, *args, **kwargs)
 |      Execute a method on the three object.
 |      
 |      Excute the method specified by `method_name` on the three
 |      object, with arguments `args`. `kwargs` is currently ignored.
 |  
 |  ---------------------------------

In [68]:
help(renderer)

Help on Renderer in module pythreejs.core.Renderer object:

class Renderer(pythreejs._base.renderable.RenderableWidget)
 |  Renderer(scene, camera, controls=None, *, width=200, height=200, background='black', background_opacity=1.0, **kwargs)
 |  
 |  Renderer
 |  
 |  Method resolution order:
 |      Renderer
 |      pythreejs._base.renderable.RenderableWidget
 |      ipywidgets.widgets.domwidget.DOMWidget
 |      ipywidgets.widgets.widget.Widget
 |      ipywidgets.widgets.widget.LoggingHasTraits
 |      traitlets.traitlets.HasTraits
 |      traitlets.traitlets.HasDescriptors
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, scene, camera, controls=None, antialias=False, alpha=False, webgl_version=2, **kwargs)
 |      Public constructor
 |  
 |  freeze(self)
 |  
 |  hold(self)
 |  
 |  render(self, scene, camera)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  background
 |  
 |  

In [44]:
from pythreejs import *
import ipywebrtc
import ipywidgets as widgets
from ipywebrtc import ImageRecorder, VideoStream
from ipywidgets import Output, VBox

view_width = 600
view_height = 400

sphere = Mesh(
    SphereBufferGeometry(1, 32, 16),
    MeshStandardMaterial(color='red')
)

cube = Mesh(
    BoxBufferGeometry(1, 1, 1),
    MeshPhysicalMaterial(color='green'),
    position=[2, 0, 4]
)

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[sphere, cube, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)
renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [45]:
stream = ipywebrtc.WidgetStream(widget=renderer, max_fps=5)
image_recorder = ImageRecorder(stream=stream,filename='test', format='png')
image_recorder.recording = True

image_recorder.autosave = True

asdf = image_recorder.download()

image_recorder.image.height
stream

WidgetStream(max_fps=5, widget=Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), proje…